In [1]:
import os
import torch
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from feature.scalers import ranged_scaler
from feature.engineering import *
from datetime import datetime, timedelta
#from mpge.rca import mpge_root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

from CARAT.model_utils import *
from CARAT.model import CausalGraphVAE

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, GATConv
from torch_geometric.utils import dense_to_sparse
from torch.distributions import Normal, Laplace, RelaxedOneHotCategorical
from torchdiffeq import odeint  # For continuous-time normalizing flows
from CARAT.components import *
from CARAT.model_utils import *
torch.backends.cudnn.benchmark = False
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

from utils.utils import set_seed
from utils.utils import logger

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)
def gxn():
    warnings.warn("RuntimeWarning", RuntimeWarning)

RuntimeWarning

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Or if you are using > Python 3.11:
with warnings.catch_warnings(action="ignore"):
    fxn()
    gxn()

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
torch.backends.cudnn.benchmark=False
torch.autograd.profiler.profile(enabled=False)
torch.autograd.profiler.emit_nvtx(enabled=False)
torch.autograd.set_detect_anomaly(mode=False)

In [3]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
print(cats_df.shape)
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

(5000000, 20)


In [4]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [5]:
cats_rows_list = metadata.rows(named=True)
cats_df.shape

(5000000, 21)

In [6]:
cats_df = make_stationary(cats_df)

⚠️ Skipping column timestamp (not a float column)
Column: aimp | Final ADF: -2211.6294 | p-value: 0.0000 | Diffs: 0
Column: amud | Final ADF: -8.0091 | p-value: 0.0000 | Diffs: 0
Column: arnd | Final ADF: -18.1248 | p-value: 0.0000 | Diffs: 0
Column: asin1 | Final ADF: -5.7565 | p-value: 0.0000 | Diffs: 1
Column: asin2 | Final ADF: -39.8976 | p-value: 0.0000 | Diffs: 3
Column: adbr | Final ADF: -111.5305 | p-value: 0.0000 | Diffs: 0
Column: adfl | Final ADF: -217.2031 | p-value: 0.0000 | Diffs: 0
Column: bed1 | Final ADF: -35.0214 | p-value: 0.0000 | Diffs: 0
Column: bed2 | Final ADF: -44.5328 | p-value: 0.0000 | Diffs: 0
Column: bfo1 | Final ADF: -8.5614 | p-value: 0.0000 | Diffs: 0
Column: bfo2 | Final ADF: -13.3015 | p-value: 0.0000 | Diffs: 0
Column: bso1 | Final ADF: -2164.1026 | p-value: 0.0000 | Diffs: 1
Column: bso2 | Final ADF: -13.5707 | p-value: 0.0000 | Diffs: 0
Column: bso3 | Final ADF: -14.7270 | p-value: 0.0000 | Diffs: 0
Column: ced1 | Final ADF: -727.3289 | p-value: 0.

In [7]:
cats_df = cats_df.to_pandas()

In [47]:
metadata['affected'].unique().to_list()

["['ced1']", "['cso1']", "['cfo1']"]

In [8]:
potential_causes = metadata['root_cause'].unique().to_list()

In [9]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:06,0.0,0.142857,-0.444480,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000072,-0.507953,-0.716059,0.0,0.100401,-0.186461
2023-01-01 00:00:07,0.0,0.142857,-0.446078,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000083,-0.507953,-0.716059,0.0,0.100408,-0.186406
2023-01-01 00:00:08,0.0,0.142857,-0.447166,0.00002,-8.000267e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000094,-0.507953,-0.716059,0.0,0.100416,-0.186345
2023-01-01 00:00:09,0.0,0.142857,-0.442843,0.00002,-7.999823e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000104,-0.507953,-0.716059,0.0,0.100426,-0.186278
2023-01-01 00:00:10,0.0,0.142857,-0.441320,0.00002,-8.000045e-12,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,0.000113,-0.507953,-0.716059,0.0,0.100438,-0.186205


In [10]:
cats_df.shape

(4999994, 17)

In [11]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [12]:
test_df = test_df.astype('float32')

In [13]:
#device = torch.device("cuda1" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:1")


In [14]:
device

device(type='cuda', index=1)

In [15]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [16]:
cols = list(test_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

In [35]:
def scale_tensor(tensor):
  """Scales a PyTorch tensor to the range [0, 1].

  Args:
    tensor: The input tensor.

  Returns:
    A new tensor with values scaled to [0, 1].
  """
  min_val = tensor.min()
  max_val = tensor.max()
  scaled_tensor = (tensor - min_val) / (max_val - min_val)
  return scaled_tensor

In [19]:
TIME_STEPS = 3
BATCH_SIZE = 100000
hidden_dim = 500
latent_dim = 8
device = torch.device("cuda:1")
num_nodes = 17

dataset_nominal = TimeSeriesDataset(train_df, time_steps=TIME_STEPS)
dataloader_nominal = DataLoader(dataset_nominal, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

# Initialize model and optimizer
model = CausalGraphVAE(input_dim=train_df.shape[1], hidden_dim=hidden_dim,
                        latent_dim=latent_dim, num_nodes=train_df.shape[1],
                        time_steps=TIME_STEPS, prior_adj=None).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-4)

# Train on nominal data
print("Pretraining on nominal data...")
model.train_model(dataloader_nominal, optimizer, num_epochs=500, patience=20)

# Extract learned adjacency
prior_adj = model.causal_graph.edge_score_now.clone().detach()


Pretraining on nominal data...
Epoch 1: Loss = 2174905900929436928.0000
Recon Loss =1423468.1250, KL Loss = 3.9480, Sparsity Loss = 502.9736, Lagrangian Loss = 165498850990096384.0000, Likelihood Loss = 0.0000
Epoch 251: Loss = 630838877434675.2500
Recon Loss =1671.8445, KL Loss = 6.1807, Sparsity Loss = 434.0810, Lagrangian Loss = 627157835448320.0000, Likelihood Loss = 0.0000


In [34]:
prior_adj = model.causal_graph.edge_score_now.clone().detach()
scaled_prior = scale_tensor(prior_adj)
for i, row in enumerate(scaled_prior):
    for j, column in enumerate(row):
        if (j in non_causal_indices) and (i in causal_indices) & (i!=j):
            continue
        else:
            scaled_prior[i,j] = 0

In [45]:
root_rank

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
aimp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
amud,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
arnd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
asin1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
asin2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
adbr,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
adfl,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
bed1,0.200303,0.501797,0.491774,0.419287,0.369055,0.571580,0.568413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.473623,0.437199,0.313898
bed2,0.556672,0.408943,0.527579,0.538709,0.503766,0.475597,0.712675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.432411,0.603195,0.403706
bfo1,0.482994,0.502200,0.506608,0.623707,0.580026,0.670484,0.550955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.528722,0.506777,0.446574


In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, GATConv
from torch_geometric.utils import dense_to_sparse
from torch.distributions import Normal, Laplace, RelaxedOneHotCategorical
from torchdiffeq import odeint  # For continuous-time normalizing flows
from CARAT.model_utils import *
torch.backends.cudnn.benchmark = False
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

def scale_tensor(tensor):
  """Scales a PyTorch tensor to the range [0, 1].

  Args:
    tensor: The input tensor.

  Returns:
    A new tensor with values scaled to [0, 1].
  """
  min_val = tensor.min()
  max_val = tensor.max()
  scaled_tensor = (tensor - min_val) / (max_val - min_val)
  return scaled_tensor

class FourierTimeEmbedding(nn.Module):
    """Encodes time indices using sinusoidal embeddings for better temporal representation."""
    def __init__(self, embedding_dim,time_steps, max_time=100):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.max_time = max_time
        
        # Create frequency bands (log-spaced)
        self.freqs = torch.exp(torch.linspace(0, time_steps, embedding_dim//2,dtype=torch.float32)).to(device)

    def forward(self, time_indices):
        """
        Args:
            time_indices: Tensor of shape [batch_size, time_steps] containing integer time indices
        Returns:
            Embedded time representations of shape [batch_size, time_steps, embedding_dim]
        """
        time_indices = time_indices.unsqueeze(-1)  # Shape [batch_size, time_steps, 1]
        sinusoidal_in = time_indices * self.freqs  # Shape [batch_size, time_steps, embedding_dim//2]
        time_embedding = torch.cat([torch.sin(sinusoidal_in), torch.cos(sinusoidal_in)], dim=-1)
        return time_embedding  # Shape [batch_size, time_steps, embedding_dim]

class TemporalRealNVPFlow(nn.Module):
    """Time-Adaptive Normalizing Flow for Latent Confounders."""
    def __init__(self, latent_dim,input_dim):
        super().__init__()
        self.scale = nn.Linear(latent_dim // 2, latent_dim // 2,dtype=torch.float32)
        self.translate = nn.Linear(latent_dim // 2, latent_dim // 2,dtype=torch.float32)
        self.time_embedding = FourierTimeEmbedding(latent_dim,input_dim)
        self.temporal_gate = nn.GRU(latent_dim, latent_dim//2, batch_first=True,dtype=torch.float32)  # Time-aware updates
        self.time_projection = nn.Linear(input_dim,1,dtype=torch.float32)
    def forward(self, z, time_context):
        z1, z2 = z.chunk(2, dim=1)  # Split into two parts
        s = torch.sigmoid(self.scale(z1))
        t = self.translate(z1)
        z2 = s * z2 + t
        time_embed = self.time_embedding(time_context)
        # Temporal adjustment to confounders
        time_out, _ = self.temporal_gate(time_embed)
        time_out = self.time_projection(time_out.permute(0,2,1)).squeeze(2)
        z2 = z2 + time_out#.squeeze(1)  # Adjust for temporal shift
        
        return torch.cat([z1, z2], dim=1)

class TemporalCausalGraph(nn.Module):
    """
    Implements a Temporal Causal Graph (TCG) with:
    - Time-dependent adjacency matrix (instantaneous + delayed effects)
    - Adaptive normalizing flow for non-stationarity handling
    """
    def __init__(self, num_nodes, hidden_dim, latent_dim,time_steps=10, prior_adj=None, mixed_data=False, num_regimes=3):
        super(TemporalCausalGraph, self).__init__()
        self.num_nodes = num_nodes
        self.latent_dim = latent_dim
        self.time_steps = time_steps
        self.mixed_data = mixed_data  # Support for categorical + continuous

        self.pos_embedding = nn.Embedding(time_steps, hidden_dim,dtype=torch.float32)

        # Learnable adjacency matrices (instantaneous + delayed)
        self.edge_score_now = nn.Parameter(torch.randn(num_nodes, num_nodes,device=device))
        self.edge_score_lag = nn.Parameter(torch.randn(num_nodes, num_nodes,device=device))
        self.prior_adj = prior_adj if prior_adj is not None else torch.zeros(num_nodes, num_nodes,device=device)

        # Direct adjacency learning
        self.dropout = nn.Dropout(p=0.3)  # Dropout rate 30%
        self.x_projection1 = nn.Sequential(
            nn.Linear(num_nodes, hidden_dim, dtype=torch.float32),
            nn.ReLU(),
            self.dropout
        )
      
        self.self_attention = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=4, dim_feedforward=hidden_dim, dtype=torch.float32,dropout=0.2),
            num_layers=2
        )
        self.x_projection2 = nn.Linear(hidden_dim,num_nodes,dtype=torch.float32)

        # Latent Confounders Z (Time-Aware)
        
        self.temporal_flow = TemporalRealNVPFlow(latent_dim,time_steps)

        # Temporal Graph Attention Network
        self.gnn = GATv2Conv(num_nodes, hidden_dim, heads=4, concat=True, dropout=0.2).to(torch.float32)

        # Mapping from latent space to num_nodes
        self.latent_to_nodes = nn.Linear(latent_dim, num_nodes,dtype=torch.float32)

        # Likelihood Models
        self.gaussian_likelihood = nn.Sequential(
            nn.Dropout(0.2),  # Dropout before final layer
            nn.Linear(hidden_dim * 4, self.num_nodes*2, dtype=torch.float32)
        )  # Mean, Log-Variance
        if mixed_data:
            self.categorical_likelihood = nn.Linear(hidden_dim, num_nodes,dtype=torch.float32)  # Gumbel-Softmax Output

    def forward(self, X, time_context,Z):
        """ Learns causal graph over time and performs inference """
        # Compute adjacency matrices
        x = self.x_projection1(X)
        pos_indices = torch.arange(self.time_steps, device=X.device)  # [time_steps]
        X_permuted = x.permute(1, 0, 2)  # [time_steps, batch_size, num_nodes]
        pos_embedding = self.pos_embedding(pos_indices).unsqueeze(1) 
        
        X_transformed = self.self_attention(X_permuted + pos_embedding)
        x = self.x_projection2(X_transformed)


        weights_schedule = generate_decreasing_weights(3,start=0.2)
        lag_mats = []
        for i in range(0,x.shape[0]):
            if i ==0:
                lag_mats.append( replace_zero(torch.sigmoid(self.edge_score_now* weights_schedule[i])  +self.prior_adj + F.normalize(F.gumbel_softmax(torch.einsum('bk,bj->kj', x[i,:,:], x[i,:,:]) * weights_schedule[i],tau=100,eps=1e-16,))).fill_diagonal_(-1)) 
            else:
                lag_mats.append( replace_zero(torch.sigmoid(self.edge_score_lag* weights_schedule[i])  +self.prior_adj + F.normalize(F.gumbel_softmax(torch.einsum('bk,bj->kj', x[i,:,:], x[i,:,:]) * weights_schedule[i],tau=100,eps=1e-16,))).fill_diagonal_(-1)) 
        
        adj_now = scale_tensor(lag_mats[0] )  # Amplify signal
        if x.shape[0] >1:
            adj_lag = scale_tensor(torch.sum(torch.stack(lag_mats[1:]), dim=0))
        else:
            adj_lag = scale_tensor(lag_mats[1])
        adj_mat = adj_now * 0.5 + adj_lag * 0.5
       
        # Encode latent confounders with time-awareness
        #Z = self.latent_Z + torch.randn_like(self.latent_Z) * 0.1
        """Z = self.temporal_flow(Z.to(device), time_context.to(device))  # Apply time-adaptive normalizing flow
        Z = self.latent_to_nodes(Z)  # Map latent space to num_nodes

        # Temporal graph attention
        edge_index,edge_weights = dense_to_sparse( adj_mat)
        X_emb = self.gnn(Z, edge_index)
        X_emb = X_emb.view(X_emb.shape[0], -1)

        # Likelihood computation
        mean_logvar = self.gaussian_likelihood(X_emb)
        mean, log_var = torch.split(mean_logvar, mean_logvar.shape[-1] // 2, dim=-1)
        log_var = torch.clamp(log_var, -5, 2)  # Stabilization
        
        likelihood = Laplace(mean, torch.exp(0.5 * log_var))"""
        likelihood = 0
        
        return adj_now, adj_lag, likelihood

In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, GATConv
from torch_geometric.utils import dense_to_sparse
from torch.distributions import Normal, Laplace, RelaxedOneHotCategorical
from torchdiffeq import odeint  # For continuous-time normalizing flows
from CARAT.components import *
from CARAT.model_utils import *
import pandas as pd
torch.backends.cudnn.benchmark = False
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

class CausalGraphVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_nodes, time_steps=10, prior_adj=None):
        super(CausalGraphVAE, self).__init__()
        self.time_steps = time_steps
        self.num_nodes = num_nodes
        self.latent_dim = latent_dim
        self.prior_adj = prior_adj
        self.causal_graph = TemporalCausalGraph(num_nodes, hidden_dim, latent_dim, time_steps, prior_adj)
        self.alpha = torch.tensor(0.0, dtype=torch.float32, requires_grad=False, device=device)
        self.rho = torch.tensor(1.0, dtype=torch.float32, requires_grad=False, device=device)

        # Temporal-aware Encoder and Decoder
        self.encoder_rnn = nn.GRU(input_dim, hidden_dim, batch_first=True,dtype=torch.float32, device=device)
        self.mu_layer = nn.Linear(hidden_dim, latent_dim,dtype=torch.float32, device=device)
        self.logvar_layer = nn.Linear(hidden_dim, latent_dim,dtype=torch.float32, device=device)
        self.decoder_rnn = nn.GRU(latent_dim, hidden_dim, batch_first=True,dtype=torch.float32, device=device)
        self.decoder_fc = nn.Linear(hidden_dim, input_dim,dtype=torch.float32, device=device)

    def encode(self, X, time_context):
        X_enc, _ = self.encoder_rnn(X)
        mu, logvar = self.mu_layer(X_enc[:, -1, :]), self.logvar_layer(X_enc[:, -1, :])
        Z = mu + torch.randn_like(logvar) * torch.exp(0.5 * logvar)
        adj_now, adj_lag, likelihood = self.causal_graph(X, time_context,Z)  # Use temporal causal graph
        return mu, logvar, adj_now, adj_lag, likelihood

    def decode(self, Z, adj_now, adj_lag):
        Z_expanded = Z.unsqueeze(1).repeat(1, self.time_steps, 1)
        X_dec, _ = self.decoder_rnn(Z_expanded)
        X_dec = self.decoder_fc(X_dec)
        return X_dec

    def forward(self, X, time_context):
        X = X.to(torch.float32)  # Convert input X to float32
        time_context = time_context.to(torch.float32)  # Convert time context to float32

        mu, logvar, adj_now, adj_lag, likelihood = self.encode(X, time_context)
        Z = mu + torch.randn_like(logvar) * torch.exp(0.5 * logvar)
        recon_X = self.decode(Z, adj_now, adj_lag)
        return recon_X, mu, logvar, adj_now, adj_lag, likelihood

    def infer_causal_effect(self, X_data, T_data, target_variable, labels, non_causal_indices=[]):
        """Infers the top causal factors for a given target variable using counterfactual analysis."""
        
        try:
            target_idx = labels.index(target_variable, 0)
        except:
            return target_variable + " not in index"
    
        self.eval()
        n_samples = X_data.shape[0]
    
        with torch.no_grad():
            _, _, adj_now, adj_lag, _ = self.encode(X_data, T_data)
            adj_matrix = adj_now * 0.5 + adj_lag * 0.5
    
            # Compute edge strengths
            causal_strengths = adj_matrix[:, target_idx].cpu().detach().numpy()
            edge_strengths = {labels[i]: float(np.round(val.item(), 6)) for i, val in enumerate(causal_strengths) if i not in non_causal_indices}

            instantaneous_strengths = adj_now[:, target_idx].cpu().detach().numpy()
            instantaneous_edge_strengths = {}
            for i, val in enumerate(instantaneous_strengths):
                if i not in non_causal_indices:
                    instantaneous_edge_strengths[labels[i]] = float(np.round(val.item(),6))
            
            lagged_strengths = adj_lag[:, target_idx].cpu().detach().numpy()
            lagged_edge_strengths = {}
            for i, val in enumerate(lagged_strengths):
                if i not in non_causal_indices:
                    lagged_edge_strengths[labels[i]] = float(np.round(val.item(),6))

            
            counterfactual_results_positive = {}
            counterfactual_results_negative = {}
    
            for i in range(self.num_nodes):
                if i == target_idx or i in non_causal_indices:
                    continue
                
                label = labels[i]
    
                # Positive intervention: set cause variable to 1
                intervention_pos = X_data.clone()
                intervention_pos[:, :, i] = 1  
                mu, logvar, adj_now_int, adj_lag_int, likelihood = self.encode(intervention_pos, T_data)
                Z = mu + torch.randn_like(logvar, dtype=torch.float32, device=device) * torch.exp(0.5 * logvar)
                recon_X = self.decode(Z, adj_now_int, adj_lag_int)
    
                # Compute normalized deviation for positive intervention
                counterfactual_results_positive[label] = torch.abs((X_data[:, :, target_idx] - recon_X[:, :, target_idx]) / (X_data[:, :, target_idx] + 1e-6)).mean().item()
    
                # Negative intervention: set cause variable to -1
                intervention_neg = X_data.clone()
                intervention_neg[:, :, i] = -1
                mu, logvar, adj_now_int, adj_lag_int, likelihood = self.encode(intervention_neg, T_data)
                Z = mu + torch.randn_like(logvar, dtype=torch.float32, device=device) * torch.exp(0.5 * logvar)
                recon_X = self.decode(Z, adj_now_int, adj_lag_int)
    
                # Compute normalized deviation for negative intervention
                counterfactual_results_negative[label] = torch.abs((X_data[:, :, target_idx] - recon_X[:, :, target_idx]) / (X_data[:, :, target_idx] + 1e-6)).mean().item()
    
            # Compute composite counterfactual score
            counterfactual_rankings = {key: counterfactual_results_positive[key] + counterfactual_results_negative[key] for key in counterfactual_results_positive}
    
            # Sort by impact score
            sorted_keys = sorted(counterfactual_rankings, key=counterfactual_rankings.get, reverse=True)
            counterfactual_rankings = {key: counterfactual_rankings[key] for key in sorted_keys}
            sorted_keys = sorted(edge_strengths, key=edge_strengths.get, reverse=True)
            top_causes = {key: edge_strengths[key] for key in sorted_keys}
            sorted_keys = sorted(instantaneous_edge_strengths, key=instantaneous_edge_strengths.get, reverse=True)
            instantaneous_effects = {key: instantaneous_edge_strengths[key] for key in sorted_keys}
            sorted_keys = sorted(lagged_edge_strengths, key=lagged_edge_strengths.get, reverse=True)
            lagged_effects = {key: lagged_edge_strengths[key] for key in sorted_keys}
            
                
        return top_causes, counterfactual_rankings, instantaneous_effects, lagged_effects

    def loss_function(self, recon_X, X, mu, logvar, likelihood, adj_now, adj_lag, epoch, max_epochs, rho_max=30.0, alpha_max=15.0, lambda_prior=5.0):
        """Loss function including reconstruction, KL divergence, DAG penalty, and prior regularization"""
        recon_loss = F.mse_loss(recon_X, X, reduction='sum')
        beta = min(1.0, epoch / max_epochs)
        kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    
        # Regularization on adjacency matrix (pretrained `prior_adj` should remain mostly intact)
        #prior_loss = lambda_prior * (torch.norm(adj_now - self.prior_adj, p=1) + torch.norm(adj_lag - self.prior_adj, p=1))
    
        sparsity_loss = torch.norm(adj_now, p=1) + torch.norm(adj_lag, p=1)
        h_value = (notears_constraint(adj_now) + notears_constraint(adj_lag))
    
        self.rho = min(rho_max, 1.0 + (epoch / max_epochs) ** 2 * 15.0)
        self.alpha = min(alpha_max, (epoch / max_epochs) ** 2 * 5.0)
    
        #likelihood_loss = (-likelihood.log_prob(X[:, 0, :]).mean()) + 5.0
        likelihood_loss = 0
    
        lagrangian_loss = (self.alpha * h_value + 0.5 * self.rho * (h_value ** 2)) / (self.num_nodes ** 2)


        return recon_loss , kl_loss , sparsity_loss , lagrangian_loss, likelihood_loss

    def train_model(self, dataloader, optimizer, num_epochs=100, patience=10,BATCH_SIZE=64,rho_max=30.0,alpha_max=15.0):
        best_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(num_epochs):
            total_loss = 0
            for batch_idx, (X_batch, time_batch) in enumerate(dataloader):
                if X_batch.shape[0] < X_batch.shape[2]:
                    continue
                optimizer.zero_grad()
                recon_X, mu, logvar, adj_now, adj_lag, likelihood = self.forward(X_batch, time_batch)
                recon_loss , kl_loss , sparsity_loss , lagrangian_loss, likelihood_loss = self.loss_function(recon_X, X_batch, mu, logvar,likelihood, adj_now, 
                                                                                            adj_lag, epoch, num_epochs,rho_max,alpha_max)
                loss = recon_loss + kl_loss + sparsity_loss + lagrangian_loss # + likelihood_loss
                
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            self.adj_mat = scale_tensor(adj_now * 0.5 + adj_lag * 0.5)

            avg_loss = total_loss / len(dataloader)
            if epoch % 250 == 0:
                print(f"Epoch {epoch + 1}: Loss = {avg_loss:.4f}")
                print(f"Recon Loss ={recon_loss:.4f}, KL Loss = {kl_loss:.4f}, Sparsity Loss = {sparsity_loss:.4f}, Lagrangian Loss = {lagrangian_loss:.4f}, Likelihood Loss = {likelihood_loss:.4f}") 

            
            # Early stopping
            if avg_loss < best_loss:
                best_loss = avg_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered. Last Epoch: " + str(epoch) )
                    print(f"Recon Loss ={recon_loss:.4f}, KL Loss = {kl_loss:.4f}, Sparsity Loss = {sparsity_loss:.4f}, Lagrangian Loss = {lagrangian_loss:.4f}, Likelihood Loss = {likelihood_loss:.4f}") 
                    break

In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from statsmodels.tsa.stattools import adfuller
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe, time_steps=10):
        """
        Args:
            dataframe (pd.DataFrame): Time-series data
            time_steps (int): Number of past time steps to consider
        """
        self.data = torch.tensor(dataframe.values, dtype=torch.float32)
        self.time_steps = time_steps
    
    def __len__(self):
        return len(self.data) - self.time_steps

    def __getitem__(self, idx):
        """
        Returns:
            X: Past `time_steps` values (shape: [time_steps, num_features])
            y: Next step prediction target (shape: [num_features])
        """
        X = self.data[idx : idx + self.time_steps, :].to(device)
        time_context = torch.arange(self.time_steps).float().to(device)  # Time indexing
        
        return X, time_context

In [80]:
cols = list(test_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

num_nodes = len(train_df.columns)

new_metadata = []
A0 = get_adjacency(cols,causal_indices,non_causal_indices,num_nodes)    
BATCH_SIZE = 64
hidden_dim=64
latent_dim=8

set_seed()
edge_correct = 0
instantaneous_correct = 0
lagged_correct = 0
counterfactual_correct = 0 
root_rank_correct = 0
total_correct = 0
total_checked = 0
incorrect = []

for i, row in enumerate(cats_rows_list):
    total_checked +=1 
    logger.info('Model: '+ str(i))
    anomaly = eval(row['affected'])[0]
    print('Anomaly: ' + anomaly)
    anomaly_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    #start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    root_cause = row['root_cause']
    #start_len = mod_df.shape[0]
    #if start_len >1000:
        #start_len = 1000
    start_len = 50
    start_time = anomaly_time- timedelta(seconds=start_len)
    finish_time = end_time + timedelta(seconds=start_len)
    mod_df = test_df[(test_df.index>= start_time) & (test_df.index<= finish_time)]
    mod_df = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1']]

    """
    FIND THE OPTIMAL NUMBER OF LAGS
    """
    TIME_STEPS = most_frequent(find_optimal_lags_for_dataframe(mod_df))+1

    dataset = TimeSeriesDataset(mod_df, time_steps=TIME_STEPS)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
    X_data = torch.empty(0,device=device)
    T_data = torch.empty(0,device=device)
    for batch_idx, (X_batch, time_batch) in enumerate(dataloader):
        X_data = torch.cat((X_data[:batch_idx], X_batch, X_data[batch_idx:]))
        T_data = torch.cat((T_data[:batch_idx], time_batch, T_data[batch_idx:]))
    
    fine_tuned = CausalGraphVAE(input_dim=num_nodes, hidden_dim=hidden_dim,
                           latent_dim=latent_dim, num_nodes=num_nodes,
                           time_steps=TIME_STEPS,prior_adj=scaled_prior).to(device)
    optimizer = torch.optim.AdamW(fine_tuned.parameters(), lr=0.01, weight_decay=1e-2)  # L2 Regularization

    
    fine_tuned.train_model(dataloader, optimizer, num_epochs=2500, patience=50,BATCH_SIZE=BATCH_SIZE,rho_max=10.0,alpha_max=2.0)

    causes, counterfactual_rankings, instantaneous_effects, lagged_effects = fine_tuned.infer_causal_effect(X_data.to(torch.float32).to(device),T_data.to(torch.float32).to(device),anomaly,cols,non_causal_indices=curr_non_causal)
    
    causes_df = pd.DataFrame.from_dict(causes, columns=['causes'],orient='index')
    causes_df=(causes_df-causes_df.min())/(causes_df.max()-causes_df.min())
    isntantaneous_df = pd.DataFrame.from_dict(instantaneous_effects, columns=['instantaneous'],orient='index')
    isntantaneous_df=(isntantaneous_df-isntantaneous_df.min())/(isntantaneous_df.max()-isntantaneous_df.min())
    lagged_df = pd.DataFrame.from_dict(lagged_effects, columns=['lagged'],orient='index')
    lagged_df=(lagged_df-lagged_df.min())/(lagged_df.max()-lagged_df.min())
    
    counterfactual_df = pd.DataFrame.from_dict(counterfactual_rankings, columns=['counterfactuals'],orient='index')
    counterfactual_df=(counterfactual_df-counterfactual_df.min())/(counterfactual_df.max()-counterfactual_df.min())
    
    total_score = pd.concat([causes_df, isntantaneous_df,lagged_df,counterfactual_df], axis=1)
    total_score['causal_strength'] = total_score.mean(axis=1)
    total_score=total_score.sort_values(by='causal_strength',ascending=False)
    
    edge_cause_1 = causes_df.sort_values(by='causes',ascending=False)[0:3].index[0]
    edge_cause_2 = causes_df.sort_values(by='causes',ascending=False)[0:3].index[1]
    edge_cause_3 = causes_df.sort_values(by='causes',ascending=False)[0:3].index[2]
    
    instant_cause_1 = isntantaneous_df.sort_values(by='instantaneous',ascending=False)[0:3].index[0]
    instant_cause_2 = isntantaneous_df.sort_values(by='instantaneous',ascending=False)[0:3].index[1]
    instant_cause_3 = isntantaneous_df.sort_values(by='instantaneous',ascending=False)[0:3].index[2]
    
    lag_cause_1 = lagged_df.sort_values(by='lagged',ascending=False)[0:3].index[0]
    lag_cause_2 = lagged_df.sort_values(by='lagged',ascending=False)[0:3].index[1]
    lag_cause_3 = lagged_df.sort_values(by='lagged',ascending=False)[0:3].index[2]
    
    counterfactual_cause_1 = counterfactual_df.sort_values(by='counterfactuals',ascending=False)[0:3].index[0]
    counterfactual_cause_2 = counterfactual_df.sort_values(by='counterfactuals',ascending=False)[0:3].index[1]
    counterfactual_cause_3 = counterfactual_df.sort_values(by='counterfactuals',ascending=False)[0:3].index[2]
    
    total_score_cause_1=total_score.sort_values(by='causal_strength',ascending=False)[0:3].index[0]
    total_score_cause_2=total_score.sort_values(by='causal_strength',ascending=False)[0:3].index[1]
    total_score_cause_3=total_score.sort_values(by='causal_strength',ascending=False)[0:3].index[2]
    
    if root_cause == edge_cause_1:
        row['edge_cause_1'] = 1
    if root_cause == edge_cause_1:
        row['edge_cause_2'] = 1
    if root_cause == edge_cause_1:
        row['edge_cause_3'] = 1
    
    if root_cause == counterfactual_cause_1:
        row['counterfactual_cause_1'] = 1
    if root_cause == counterfactual_cause_2:
        row['counterfactual_cause_2'] = 1
    if root_cause == counterfactual_cause_3:
        row['counterfactual_cause_3'] = 1
    
    
    if root_cause == total_score_cause_1:
        row['total_score_cause_1'] = 1
    if root_cause == total_score_cause_2:
        row['total_score_cause_2'] = 1
    if root_cause == total_score_cause_3:
        row['total_score_cause_3'] = 1
    
    if root_cause == instant_cause_1:
        row['instant_cause_1'] = 1
    if root_cause == instant_cause_2:
        row['instant_cause_2'] = 1
    if root_cause == instant_cause_3:
        row['instant_cause_3'] = 1
    
    if root_cause == lag_cause_1:
        row['lag_cause_1'] = 1
    if root_cause == lag_cause_2:
        row['lag_cause_2'] = 1
    if root_cause == lag_cause_3:
        row['lag_cause_3'] = 1
    
    new_metadata.append(row)
    
    if root_cause in [total_score_cause_1 , total_score_cause_2 , total_score_cause_3]:
        total_correct+=1
    if root_cause in [edge_cause_1 , edge_cause_2 , edge_cause_3]:
        edge_correct+=1
    if root_cause in [counterfactual_cause_1 , counterfactual_cause_2 , counterfactual_cause_3]:
        counterfactual_correct+=1
    if root_cause in [instant_cause_1 , instant_cause_2 , instant_cause_3]:
        instantaneous_correct+=1
    if root_cause in [lag_cause_1 , lag_cause_2 , lag_cause_3]:
        lagged_correct+=1
    
    total_accuracy = total_correct/total_checked* 100
    edge_accuracy = edge_correct/total_checked* 100
    cf_accuracy = counterfactual_correct/total_checked* 100
    instant_accuracy = instantaneous_correct/total_checked* 100
    lag_accuracy = lagged_correct/total_checked* 100
    
    if root_cause not in [total_score_cause_1 , total_score_cause_2 , total_score_cause_3,edge_cause_1 , edge_cause_2 , edge_cause_3 ]:
        incorrect.append(i) 
    logger.info(f"Edge Accuracy = {edge_accuracy:.2f}, Instantaneous Accuracy = {instant_accuracy:.2f}, Lagged Accuracy = {lag_accuracy:.2f}, Counterfactual Accuracy = {cf_accuracy:.2f},  Blended Accuracy = {total_accuracy:.2f}  ") 



2025-03-13 10:53:59,097 INFO -- Model: 0
/opt/data/home/j.lowhorn/miniconda3/envs/rca/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:702: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]


Anomaly: cfo1
Epoch 1: Loss = 209954475.7500
Recon Loss =70.2668, KL Loss = 0.8316, Sparsity Loss = 465.2918, Lagrangian Loss = 40883288.0000, Likelihood Loss = 0.0000
Epoch 251: Loss = 86057.8305
Recon Loss =1.2512, KL Loss = 0.7205, Sparsity Loss = 399.6961, Lagrangian Loss = 97666.7578, Likelihood Loss = 0.0000
Epoch 501: Loss = 55293.7184
Recon Loss =1.0454, KL Loss = 0.4710, Sparsity Loss = 390.7825, Lagrangian Loss = 55512.2188, Likelihood Loss = 0.0000
Epoch 751: Loss = 26403.0002
Recon Loss =0.3202, KL Loss = 0.5269, Sparsity Loss = 379.0124, Lagrangian Loss = 26120.6387, Likelihood Loss = 0.0000
Epoch 1001: Loss = 10829.5494
Recon Loss =0.7400, KL Loss = 0.5056, Sparsity Loss = 364.8170, Lagrangian Loss = 10475.9697, Likelihood Loss = 0.0000
Epoch 1251: Loss = 4441.0924
Recon Loss =0.2602, KL Loss = 0.4937, Sparsity Loss = 349.4449, Lagrangian Loss = 4091.5305, Likelihood Loss = 0.0000
Epoch 1501: Loss = 1978.1325
Recon Loss =0.4989, KL Loss = 0.5233, Sparsity Loss = 333.0504,

2025-03-13 11:07:39,785 INFO -- Edge Accuracy = 0.00, Instantaneous Accuracy = 0.00, Lagged Accuracy = 0.00, Counterfactual Accuracy = 100.00,  Blended Accuracy = 100.00  
2025-03-13 11:07:39,785 INFO -- Model: 1


Anomaly: cfo1
Epoch 1: Loss = 25958299.9646
Recon Loss =29.2614, KL Loss = 0.6443, Sparsity Loss = 413.3897, Lagrangian Loss = 353027.3438, Likelihood Loss = 0.0000
Epoch 251: Loss = 474.4332
Recon Loss =1.9683, KL Loss = 0.9438, Sparsity Loss = 321.6603, Lagrangian Loss = 147.3307, Likelihood Loss = 0.0000
Epoch 501: Loss = 145.4575
Recon Loss =1.0885, KL Loss = 0.8156, Sparsity Loss = 133.1582, Lagrangian Loss = 9.4888, Likelihood Loss = 0.0000


2025-03-13 11:30:28,950 INFO -- Edge Accuracy = 50.00, Instantaneous Accuracy = 50.00, Lagged Accuracy = 50.00, Counterfactual Accuracy = 50.00,  Blended Accuracy = 50.00  
2025-03-13 11:30:28,951 INFO -- Model: 2


...

2025-03-13 11:50:32,253 INFO -- Edge Accuracy = 33.33, Instantaneous Accuracy = 33.33, Lagged Accuracy = 33.33, Counterfactual Accuracy = 33.33,  Blended Accuracy = 33.33  
2025-03-13 11:50:32,253 INFO -- Model: 3
/opt/data/home/j.lowhorn/miniconda3/envs/rca/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:702: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]


Anomaly: ced1
Epoch 1: Loss = 210437159.5714
Recon Loss =93.2553, KL Loss = 0.3928, Sparsity Loss = 445.7740, Lagrangian Loss = 8849002.0000, Likelihood Loss = 0.0000
Epoch 251: Loss = 143552.0000
Recon Loss =1.4146, KL Loss = 0.6759, Sparsity Loss = 395.6437, Lagrangian Loss = 166953.6875, Likelihood Loss = 0.0000
Epoch 501: Loss = 107155.6562
Recon Loss =0.3918, KL Loss = 0.4801, Sparsity Loss = 389.9657, Lagrangian Loss = 124433.1250, Likelihood Loss = 0.0000
Epoch 751: Loss = 72575.3281
Recon Loss =0.5963, KL Loss = 0.4241, Sparsity Loss = 382.3292, Lagrangian Loss = 84183.7891, Likelihood Loss = 0.0000
Epoch 1001: Loss = 44358.2182
Recon Loss =0.6485, KL Loss = 0.4478, Sparsity Loss = 373.3398, Lagrangian Loss = 51311.7383, Likelihood Loss = 0.0000
Epoch 1251: Loss = 25801.5198
Recon Loss =0.1829, KL Loss = 0.4177, Sparsity Loss = 363.6976, Lagrangian Loss = 29693.4922, Likelihood Loss = 0.0000
Epoch 1501: Loss = 14853.7497
Recon Loss =0.5717, KL Loss = 0.4281, Sparsity Loss = 353

2025-03-13 11:59:27,314 INFO -- Edge Accuracy = 25.00, Instantaneous Accuracy = 25.00, Lagged Accuracy = 25.00, Counterfactual Accuracy = 50.00,  Blended Accuracy = 50.00  
2025-03-13 11:59:27,315 INFO -- Model: 4
/opt/data/home/j.lowhorn/miniconda3/envs/rca/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:702: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]


Anomaly: cfo1
Epoch 1: Loss = 300613802.5714
Recon Loss =150.2297, KL Loss = 0.3903, Sparsity Loss = 452.7309, Lagrangian Loss = 18160962.0000, Likelihood Loss = 0.0000
Epoch 251: Loss = 181155.0580
Recon Loss =2.1790, KL Loss = 1.4071, Sparsity Loss = 396.0241, Lagrangian Loss = 210631.1406, Likelihood Loss = 0.0000
Epoch 501: Loss = 137378.3750
Recon Loss =1.0847, KL Loss = 1.1319, Sparsity Loss = 390.5897, Lagrangian Loss = 159648.8281, Likelihood Loss = 0.0000
Epoch 751: Loss = 93125.6183
Recon Loss =0.6888, KL Loss = 1.0500, Sparsity Loss = 382.9973, Lagrangian Loss = 108113.2109, Likelihood Loss = 0.0000
Epoch 1001: Loss = 56770.0190
Recon Loss =0.6413, KL Loss = 0.8919, Sparsity Loss = 374.0209, Lagrangian Loss = 65757.6797, Likelihood Loss = 0.0000
Epoch 1251: Loss = 32891.1557
Recon Loss =0.7276, KL Loss = 1.1700, Sparsity Loss = 364.4062, Lagrangian Loss = 37950.9102, Likelihood Loss = 0.0000
Epoch 1501: Loss = 18829.7369
Recon Loss =0.3520, KL Loss = 0.9282, Sparsity Loss = 

KeyboardInterrupt: 